## File search assistant
The McKessonMDTranscripts demonstrates the creation and execution of an assistant using the file_search feature with a knowledge base based on earnings call transcripts downloaded from the internet.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.8.8"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.8.8

## Ensure the McKessonMDTranscripts assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\McKessonMDTranscripts" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"McKessonMDTranscripts" contains the following files:
```
│   manifest.json
│   prompt.md
│
└───VectorStores
    └───McKessonMDTranscripts
            211101-MCK-Q2FY22-Earnings-Call-Transcript.md
            220202-MCK-Q3FY22-Earnings-Call-Transcript.md
            220505-MCK-Q4FY22-Earnings-Call-Transcript.md
            220803-MCK-Q1FY23-Earnings-Call-Transcript.md
            230508-MCK-Q4FY23-Earnings-Call-Transcript.md
            230802-MCK-Q1FY24-Earnings-Call-Transcript.md
            231101-MCK-Q2FY24-Earnings-Call-Transcript.md
            MCK-Q1FY21-Transcript.md
            MCK-Q1FY22-Transcript.md
            MCK-Q2-FY23-Earnings-Transcript.md
            MCK-Q2FY21-Transcript.md
            MCK-Q3-FY24-Earnings-Transcript.md
            MCK-Q4-FY24-Earnings-Call-Transcript.md
            MCK-Q4FY21-Transcript.md
            MCK-US-20230201-2761065-C.md
            Q1-FY19-Earnings-Call-Transcript.md
            Q1-FY20-Earnings-Call-Transcript.md
            Q2-FY19-Earnings-Call-Transcript.md
            Q2-FY20-Earnings-Call-Transcript.md
            Q3-FY19-Earnings-Call-Transcript.md
            Q3FY20-MCK-Earnings-Call-Transcript.md
            Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md
            Q4-FY19-Earnings-Call-Transcript.md
            Q4FY20-MCK-Corrected-Transcript.md
```

### Explanation
`AssistantUtility.Create` will create the vector store if necessary, upload the files, and then update the manifest with the vector store ID before creating the assistant.

In [2]:
var assistantId = await AssistantUtility.Create("McKessonMDTranscripts", config);
Console.WriteLine(assistantId)

asst_QHdqXTGsCxCDGWsvb1px2sAb


## Run the Assistant

`output.Dialog` shows the conversation.

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "McKessonMDTranscripts",
    Instructions = "How did covid impact results over time?",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: How did covid impact results over time?

Assistant: I searched my knowledge base for the answer.

The impact of COVID-19 on McKesson's results over time can be summarized as follows:

1. **Initial Impact and Fiscal 2020**:
   - In the fourth quarter of fiscal 2020, McKesson experienced increased demand for pharmaceuticals and personal protective equipment (PPE) due to the pandemic, resulting in approximately $2 billion of incremental revenue. This was partially offset by higher operating expenses related to frontline employee support and remote workforce needs【4:0†source】【4:3†source】【4:13†source】.

2. **Fiscal 2021**:
   - Throughout fiscal 2021, McKesson navigated significant uncertainties due to COVID-19. The company saw volume declines in the early part of the fiscal year, but recovery began earlier than anticipated. The distribution of COVID-19 vaccines and ancillary supply kits contributed positively to results in the later part of the year【4:1†source】【4:4†source】【4:9†source】【4:10†source】.

3. **Fiscal 2022**:
   - The results were influenced by the ongoing distribution of COVID-19 vaccines and tests. However, there were challenges such as a light cold and flu season and variability in COVID-19 demand, which impacted results. Despite these challenges, McKesson continued to support its customers with necessary supplies and services【4:5†source】【4:6†source】【4:12†source】.

4. **Fiscal 2023**:
   - The benefit from COVID-19-related activities, including vaccine distribution and test kits, started to decline compared to the previous years. The company noted a reduction in sales of COVID-19 tests and lower contributions from government contracts for vaccine distribution. However, McKesson continued to see growth in its core businesses, including primary care and specialty products【4:2†source】【4:8†source】【4:14†source】.

Overall, McKesson's response to the pandemic involved adapting to fluctuating demand, supporting healthcare providers with essential supplies, and leveraging its distribution capabilities to aid in vaccine distribution. The impact of COVID-19 evolved from initial demand spikes and logistical challenges to more stable contributions from ongoing pandemic-related activities, with a gradual return to focusing on core business growth.


In [4]:
output.LastMessage.DisplayAs("text/markdown");

The impact of COVID-19 on McKesson's results over time can be summarized as follows:

1. **Initial Impact and Fiscal 2020**:
   - In the fourth quarter of fiscal 2020, McKesson experienced increased demand for pharmaceuticals and personal protective equipment (PPE) due to the pandemic, resulting in approximately $2 billion of incremental revenue. This was partially offset by higher operating expenses related to frontline employee support and remote workforce needs【4:0†source】【4:3†source】【4:13†source】.

2. **Fiscal 2021**:
   - Throughout fiscal 2021, McKesson navigated significant uncertainties due to COVID-19. The company saw volume declines in the early part of the fiscal year, but recovery began earlier than anticipated. The distribution of COVID-19 vaccines and ancillary supply kits contributed positively to results in the later part of the year【4:1†source】【4:4†source】【4:9†source】【4:10†source】.

3. **Fiscal 2022**:
   - The results were influenced by the ongoing distribution of COVID-19 vaccines and tests. However, there were challenges such as a light cold and flu season and variability in COVID-19 demand, which impacted results. Despite these challenges, McKesson continued to support its customers with necessary supplies and services【4:5†source】【4:6†source】【4:12†source】.

4. **Fiscal 2023**:
   - The benefit from COVID-19-related activities, including vaccine distribution and test kits, started to decline compared to the previous years. The company noted a reduction in sales of COVID-19 tests and lower contributions from government contracts for vaccine distribution. However, McKesson continued to see growth in its core businesses, including primary care and specialty products【4:2†source】【4:8†source】【4:14†source】.

Overall, McKesson's response to the pandemic involved adapting to fluctuating demand, supporting healthcare providers with essential supplies, and leveraging its distribution capabilities to aid in vaccine distribution. The impact of COVID-19 evolved from initial demand spikes and logistical challenges to more stable contributions from ongoing pandemic-related activities, with a gradual return to focusing on core business growth.

## Annotations

In [5]:
Console.WriteLine(JsonSerializer.Serialize(output.Annotations, new JsonSerializerOptions() { WriteIndented = true }));

[
  {
    "type": "file_citation",
    "text": "\u30104:0\u2020source\u3011",
    "start_index": 468,
    "end_index": 480,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:3\u2020source\u3011",
    "start_index": 480,
    "end_index": 492,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:13\u2020source\u3011",
    "start_index": 492,
    "end_index": 505,
    "file_citation": {
      "file_id": "assistant-v1vgOsajg7zOoaHPJaHKvA3i",
      "file_name": "",
      "quote": null
    }
  },
  {
    "type": "file_citation",
    "text": "\u30104:1\u2020source\u3011",
    "start_index": 861,
    "end_index": 873,
    "file_citation": {
      "file_id": "assistant-LtpQtNw0yqJbx07lbEJahob9",
      "file_name": "",
      "quote

### Formatting Output
The current version of the API leaves some things to be desired. First, because of chunking it can cite the same source twice. Second, 'quote' is supposed to contain a snippet, but currently, it is always null

In [6]:
var files = new Dictionary<string, string>();
var annotatedMessage = output.LastMessage;
foreach(var annotation in output.Annotations)
{
    // Only get the file name once per id
    if(!files.ContainsKey(annotation.FileCitation.FileId))
    {
        files[annotation.FileCitation.FileId] = await annotation.GetFileName(config);
    }
    // But the annotation itself may be a duplicate citation to the same file, so one must process them all
    // You can format the replacement however you like. In this case it just makes an md link
    annotatedMessage = annotatedMessage.Replace(annotation.Text, $" [{files[annotation.FileCitation.FileId]}]({files[annotation.FileCitation.FileId]})");
}

foreach(var file in files)
{
    Console.WriteLine($"{file.Key} = {file.Value}");
}
Console.WriteLine("-------------------");
annotatedMessage.DisplayAs("text/markdown");

assistant-v1vgOsajg7zOoaHPJaHKvA3i = McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md
assistant-LtpQtNw0yqJbx07lbEJahob9 = McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md
assistant-mLYhwg7I9ywfWQPh1xEf1IXY = McKessonMDTranscripts_MCK-Q4FY21-Transcript.md
assistant-QaQenndgqbFTt9oZunYGcfv3 = McKessonMDTranscripts_MCK-Q1FY21-Transcript.md
assistant-KVLQgIZo6R8UzDv01PXDkrR6 = McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md
assistant-fslYRzYxsqiFwPGd7Fns83se = McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md
assistant-93mnSr4UmBc6zfhnwJLoncV0 = McKessonMDTranscripts_220803-MCK-Q1FY23-Earnings-Call-Transcript.md
-------------------


The impact of COVID-19 on McKesson's results over time can be summarized as follows:

1. **Initial Impact and Fiscal 2020**:
   - In the fourth quarter of fiscal 2020, McKesson experienced increased demand for pharmaceuticals and personal protective equipment (PPE) due to the pandemic, resulting in approximately $2 billion of incremental revenue. This was partially offset by higher operating expenses related to frontline employee support and remote workforce needs [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md) [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md) [McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md](McKessonMDTranscripts_Q4FY20-MCK-Corrected-Transcript.md).

2. **Fiscal 2021**:
   - Throughout fiscal 2021, McKesson navigated significant uncertainties due to COVID-19. The company saw volume declines in the early part of the fiscal year, but recovery began earlier than anticipated. The distribution of COVID-19 vaccines and ancillary supply kits contributed positively to results in the later part of the year [McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md](McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md) [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md) [McKessonMDTranscripts_MCK-Q1FY21-Transcript.md](McKessonMDTranscripts_MCK-Q1FY21-Transcript.md) [McKessonMDTranscripts_MCK-Q4FY21-Transcript.md](McKessonMDTranscripts_MCK-Q4FY21-Transcript.md).

3. **Fiscal 2022**:
   - The results were influenced by the ongoing distribution of COVID-19 vaccines and tests. However, there were challenges such as a light cold and flu season and variability in COVID-19 demand, which impacted results. Despite these challenges, McKesson continued to support its customers with necessary supplies and services [McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md](McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md) [McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md](McKessonMDTranscripts_220505-MCK-Q4FY22-Earnings-Call-Transcript.md) [McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md](McKessonMDTranscripts_Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md).

4. **Fiscal 2023**:
   - The benefit from COVID-19-related activities, including vaccine distribution and test kits, started to decline compared to the previous years. The company noted a reduction in sales of COVID-19 tests and lower contributions from government contracts for vaccine distribution. However, McKesson continued to see growth in its core businesses, including primary care and specialty products [McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md](McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md) [McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md](McKessonMDTranscripts_230508-MCK-Q4FY23-Earnings-Call-Transcript.md) [McKessonMDTranscripts_220803-MCK-Q1FY23-Earnings-Call-Transcript.md](McKessonMDTranscripts_220803-MCK-Q1FY23-Earnings-Call-Transcript.md).

Overall, McKesson's response to the pandemic involved adapting to fluctuating demand, supporting healthcare providers with essential supplies, and leveraging its distribution capabilities to aid in vaccine distribution. The impact of COVID-19 evolved from initial demand spikes and logistical challenges to more stable contributions from ongoing pandemic-related activities, with a gradual return to focusing on core business growth.

## Clean Up

In [7]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "McKessonMDTranscripts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
